# Drift-Free Joint Kinematics from Dual IMUs

This notebook demonstrates the two estimation algorithms from [Weygers & Kok (2020)](https://ieeexplore.ieee.org/document/9044292/):

- **MEKF-acc**: Online Multiplicative Extended Kalman Filter
- **MAP-acc**: Offline Maximum A Posteriori optimization (Gauss-Newton)

Both estimate the relative orientation between two body segments using dual IMU measurements.

In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import time

from dfjimu import mekf_acc, map_acc
from dfjimu.utils.common import quatmultiply, quatconj, angDist

## Load dataset

Each `.mat` file contains synchronised accelerometer and gyroscope data from two IMUs, ground-truth orientations, and the position vectors from each sensor to the joint center.

In [ ]:
mat = scipy.io.loadmat('../data/data_2D_07.mat', squeeze_me=True, struct_as_record=False)
data = mat['data']

sensor_data = data.sensorData
acc1 = sensor_data[:, 0:3]; gyr1 = sensor_data[:, 3:6]
acc2 = sensor_data[:, 6:9]; gyr2 = sensor_data[:, 9:12]

# r1, r2 are optional — mekf_acc/map_acc will auto-estimate them if omitted
r1 = np.atleast_1d(-data.r_12).flatten()
r2 = np.atleast_1d(-data.r_21).flatten()

Fs = 50.0
q_init = np.array([1.0, 0.0, 0.0, 0.0])

# Ground-truth relative orientation
qGS1_ref = data.ref[:, 0:4]
qGS2_ref = data.ref[:, 4:8]
q_ref = quatmultiply(quatconj(qGS1_ref), qGS2_ref)

N = sensor_data.shape[0]
t = np.arange(N) / Fs
print(f'Dataset: {N} samples, {N/Fs:.1f} s at {Fs:.0f} Hz')

## Run MEKF-acc (online filtering)

In [ ]:
start = time.time()
q1_mekf, q2_mekf = mekf_acc(gyr1, gyr2, acc1, acc2, r1, r2, Fs, q_init)
time_mekf = time.time() - start

q_rel_mekf = quatmultiply(quatconj(q1_mekf), q2_mekf)
print(f'MEKF-acc: {time_mekf:.4f} s')

## Run MAP-acc (optimization-based smoothing)

In [ ]:
cov_w = np.eye(6) * 0.01
cov_i = np.eye(3) * 0.1
cov_lnk = np.eye(3)

start = time.time()
q1_map, q2_map = map_acc(gyr1, gyr2, acc1, acc2, r1, r2, Fs, q_init,
                         cov_w, cov_i, cov_lnk, iterations=10)
time_map = time.time() - start

q_rel_map = quatmultiply(quatconj(q1_map), q2_map)
print(f'MAP-acc:  {time_map:.4f} s')

## Auto-estimated lever arms

When `r1` and `r2` are not available (e.g. no CAD model), they can be omitted. Both `mekf_acc` and `map_acc` will automatically estimate them from the accelerometer data using `estimate_lever_arms()`.

In [ ]:
# Run MEKF-acc without providing r1, r2
q1_auto, q2_auto = mekf_acc(gyr1, gyr2, acc1, acc2, Fs=Fs, q_init=q_init)

q_rel_auto = quatmultiply(quatconj(q1_auto), q2_auto)
err_auto = np.array([angDist(q_ref[i], q_rel_auto[i]) for i in range(N)]).flatten()
rmse_auto = np.sqrt(np.mean(err_auto**2))
print(f'MEKF-acc (auto r1,r2): RMSE = {rmse_auto:.2f} deg')

## Compare relative orientation

Convert relative quaternions to Euler angles for visualisation. For this 2D dataset, the dominant rotation is about a single axis.

In [ ]:
def quat_to_euler_xyz(q):
    """Convert quaternion [w,x,y,z] array to Euler angles (x,y,z) in degrees."""
    q = np.atleast_2d(q)
    w, x, y, z = q[:, 0], q[:, 1], q[:, 2], q[:, 3]
    
    # Roll (x)
    sinr = 2 * (w * x + y * z)
    cosr = 1 - 2 * (x**2 + y**2)
    roll = np.arctan2(sinr, cosr)
    
    # Pitch (y)
    sinp = 2 * (w * y - z * x)
    sinp = np.clip(sinp, -1.0, 1.0)
    pitch = np.arcsin(sinp)
    
    # Yaw (z)
    siny = 2 * (w * z + x * y)
    cosy = 1 - 2 * (y**2 + z**2)
    yaw = np.arctan2(siny, cosy)
    
    return np.degrees(np.column_stack([roll, pitch, yaw]))

euler_ref = quat_to_euler_xyz(q_ref)
euler_mekf = quat_to_euler_xyz(q_rel_mekf)
euler_map = quat_to_euler_xyz(q_rel_map)

In [ ]:
labels = ['Roll (x)', 'Pitch (y)', 'Yaw (z)']

fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True)
for i, ax in enumerate(axes):
    ax.plot(t, euler_ref[:, i], 'k-', label='Reference', linewidth=1)
    ax.plot(t, euler_mekf[:, i], '-', label='MEKF-acc', alpha=0.8, linewidth=0.8)
    ax.plot(t, euler_map[:, i], '-', label='MAP-acc', alpha=0.8, linewidth=0.8)
    ax.set_ylabel(f'{labels[i]} (deg)')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Time (s)')
fig.suptitle('Relative joint orientation — estimated vs reference', fontsize=12)
fig.tight_layout()
plt.show()

## Angular error over time

In [ ]:
# Angular distance at each timestep
err_mekf = np.array([angDist(q_ref[i], q_rel_mekf[i]) for i in range(N)]).flatten()
err_map = np.array([angDist(q_ref[i], q_rel_map[i]) for i in range(N)]).flatten()

rmse_mekf = np.sqrt(np.mean(err_mekf**2))
rmse_map = np.sqrt(np.mean(err_map**2))

fig, ax = plt.subplots(figsize=(10, 3))
ax.plot(t, err_mekf, label=f'MEKF-acc (RMSE {rmse_mekf:.2f}\u00b0)', alpha=0.8, linewidth=0.7)
ax.plot(t, err_map, label=f'MAP-acc (RMSE {rmse_map:.2f}\u00b0)', alpha=0.8, linewidth=0.7)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Angular error (deg)')
ax.set_title('Estimation error vs ground truth')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

## Summary

In [ ]:
print(f'{"Method":<12} {"RMSE (deg)":<12} {"Time (s)":<12} {"Speedup":<10}')
print('-' * 46)
print(f'{"MAP-acc":<12} {rmse_map:<12.2f} {time_map:<12.4f} {"1.0x":<10}')
print(f'{"MEKF-acc":<12} {rmse_mekf:<12.2f} {time_mekf:<12.4f} {time_map/time_mekf:.1f}x')